In [5]:
##Python .py code from .jpynb:
from pyspark.sql.functions import rand
from pyspark.sql.functions import lower
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import lit
from pyspark.sql.functions import row_number, lit, dense_rank
from pyspark.sql import functions as F, Window
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

#############################################
#            read data                      # 
#############################################
print("Read Anobii data")

# Read table which contains for each item_id (id of each book), the id of the author (author_id) who wrote that book
file = "/data/SMARTDATA/books/anobii_2021/sql/author_item.csv"
DFauthorbooks = spark.read.csv(file,header=True) #MOSTRA I LIBRI RELATIVI AGLI AUTORI

# Read table which contains for each author_id (id of each auhtor), the info of that author
file = "/data/SMARTDATA/books/anobii_2021/sql/author_display.csv"
DFdisplay = spark.read.csv(file,header=True) #MOSTRA GLI AUTORI DEL DATABASE

# Read table which contains the mapping of the language (language=11 means italian)
file = "/data/SMARTDATA/books/anobii_2021/sql/language_mapping.csv"
DFlanguages = spark.read.csv(file,header=True)

# Read table which contains the info about items, i.e. books
file = "/data/SMARTDATA/books/anobii_2021/sql/item.csv"
DFitems_anobii = spark.read.csv(file,header=True) 
# DFitems_anobii.show(1, False)

# Read table which contains the rate given by a person to a book
file = "/data/SMARTDATA/books/anobii_2021/sql/link_person_item.csv" #item_review ha il voto dato dall'utente a quel libro
DFinfo = spark.read.csv(file,header=True)
# DFinfo.show(1, False)

# Read table which contains the genre of each book
file = "anobii_genres/genres5.csv"
DFgenres = spark.read.csv(file, header=True)
#DFgenres.filter(DFgenres.itemid == "1981748").show(10, False)

file = "anobii_genres/content.csv"
DFdescriptions = spark.read.csv(file, header=True)
DFdescriptions.show(1, False)
#FUNZIONI
def title_and_subtitle(line): #Usata per splittare titolo e sottotitolo nel database delle biblioteche e renderli più simili ad anobii
    if len(line.title.split(" : ")) > 1: #Il libro possiede un sottotitolo
        title = line.title.split(" : ")[0]
        sub_title = line.title.split(" : ")[1]
        return Row(line.manifestation_id_new, line.patron_id_md5, title, sub_title, line.author, 4, "None", 4, "None", line.ISBNISSN_new, "bct") #NE APPROFITTO ANCHE PER INSERIRE VOTO E CAMPI PER IL MERGE (PER ORA NULL)
    else: #Il libro non possiede un sottotitolo
        return Row(line.manifestation_id_new, line.patron_id_md5, line.title, "None", line.author, 4, "None", 4, "None", line.ISBNISSN_new, "bct")

def create_user_dictionary(rdd): #Used to assign an integer id to each user of the rdd (to get the rows of the CSR)
    rdd = rdd.map(lambda x: (str(x.person_id), list(x))).sortByKey()
    user_dictionary = rdd.countByKey()
    i = 0
    for key in user_dictionary.keys():
        user_dictionary[key] = i
        i += 1
    return user_dictionary

def create_book_dictionary(rdd): #Used to assign an integer id to each book of the rdd (to get the column of the CSR)
    rdd = rdd.map(lambda x: (x.book_id, list(x))).sortByKey()
    book_dictionary = rdd.countByKey()
    i = 0
    for key in book_dictionary.keys():
        book_dictionary[key] = i
        i += 1
    return book_dictionary

def addDataType1(line):
    return Row(line.item_id, line.person_id, line.title, str(line.sub_title), line.item_review, line.total_review, line.average_rating, str(line.total_votes), line.isbn, "anobii")


def addRowColumnId(line): #Funzione usata per aggiungere gli indici per ogni libro e utente
    book_number = book_dictionary[line.book_id] 
    user_number = user_dictionary[line.person_id]
    return Row(line.book_id,
               line.title,
               line.sub_title,
               line.total_wishlist,
               line.no_of_page,
               line.publication_date,
               line.publisher,
               line.binding,
               line.edition,
               line.product_type,
               line.total_votes,
               line.data_type,
               line.person_id,
               line.item_review,
               line.author,
               line.genre,
               line.encrypt_item_id,
               line.isbn,
               line.total_count,
               line.average_rating,
               line.total_review,
               str(user_number), 
               str(book_number))
    return Row(line.person_id, line.title, str(line.sub_title), line.item_review, line.data_type, line.book_id, line.author, line.genre, line.encrypt_item_id, line.total_votes_or_loans, line.average_rating, line.total_review, line.isbn, str(user_number), str(book_number))

def title_and_subtitle_books(line): #FUNZIONE USATA PER DIVIDERE TITOLO E SOTTOTITOLO NEL DATASET DEI LIBRI DELLE BIBLIOTECHE E PER ELIMINARE EDITION_DATE ED EDITION_LANGUAGE
    if len(line.title.split(" : ")) > 1:
        title = line.title.split(" : ")[0]
        sub_title = line.title.split(" : ")[1]
    else:
        title = line.title
        sub_title = "None"
    #"title", "sub_title", "author", "publisher", "book_id"
    return Row(title, sub_title, line.author, line.publisher, line.manifestation_id_new, line.ISBNISSN_new)

print("Raws data have been read")
#these are the columns which are needed for our work, for the item table. The others can be dropped
DFitems_anobii_cols_to_keep = [
                        "item_id",\
                        "isbn",\
#                         "family_id",\
                        "title",\
                        "sub_title",\
                        # "barcode",\
                        # "image_source",\
                        # "image_width",\
                        # "image_height",\
                        "no_of_page",\
                        "publication_date",\
                        "publisher",\
                        "binding",\
                        "edition",\
                        # "reading_level",\
                        # "height",\
                        # "height_unit",\
                        # "length",\
                        # "length_unit",\
                        # "width",\
                        # "width_unit",\
                        # "weight",\
                        # "weight_unit",\
                        # "salesrank",\
                        # "item_popularity",\
                        # "check_same_family",\
                        # "check_internal_family",\
                        # "last_update",\
                        "average_rating",\
                        "total_review",\
                        "product_type",\
                        # "title_foreign",\
                        # "image_process",\
                        # "amazon",\
                        # "google",\
                        "encrypt_item_id",\
                        # "pid",\
                        # "binding_id",\
                        # "problem",\
                        "language",\
                        # "language_correct",\
                        # "ean",\
                        # "family_head",\
                        # "google_time",\
                        # "total_world",\
                        # "lock",\
                        # "volumes",\
                        # "publication_country_id",\
                        # "added_date",\
                        # "publishing_status",\
                        # "total_libraries",\
                        # "unavailable_date",\
                        # "table_of_contents_html",\
                        # "product_form_detail",\
                        # "total_edition_ratings",\
                        # "epub_url",\
                        # "imprint_name",\
                        # "is_sellable",\
                        # "total_topics",\
                        # "publisher_id",\
                        "total_wishlist",\
                        # "embargo_date",\
                        # "data_source",\
                        # "ebook_type",\
                        # "has_ebook",\
                        # "fulfillment_book_id",\
                        # "ebook_filesize",\
                        # "sample_status",\
                        # "sample_url",\
                        # "sample_filesize",\
                      ]
DFitems_anobii = DFitems_anobii.select(DFitems_anobii_cols_to_keep)
print("Filter Anobii data (select books in italian, drop comics and books with a low number of ratings)")
#FILTRAGGIO 1: linguaggio italiano
#Mi basterà filtrare da item.csv tutte le tuple con language diverso da 11.

DFfilteredlanguageitems = DFitems_anobii.filter(DFitems_anobii.language == "11")
DFitems_anobii = DFitems_anobii.select(DFitems_anobii_cols_to_keep)
#DFfilteredlanguageitems.show(20, False)

#FILTRAGGIO 2: solo "hardcover" e "paperback"

DFfilteredbindingitems = DFfilteredlanguageitems.filter((DFfilteredlanguageitems.binding == "Paperback") | (DFfilteredlanguageitems.binding == "Hardcover"))
#DFfilteredbindingitems.show(20, False)

#FILTRAGGIO 3: eliminare i fumetti (per quanto possibile)
#Userò come parole chiave personaggi famosi Disney quali Topolino e Paperino e i fumetti Bonelli (che fra gli italiani
#sono probabilmente i più gettonati)

DFfilteredmagazineitems = DFfilteredbindingitems.filter(~(DFfilteredbindingitems.title.contains("Topolino")) | (DFfilteredbindingitems.title.contains("Paperino"))|(DFfilteredbindingitems.title.contains("Tex"))|(DFfilteredbindingitems.title.contains("Dylan Dog"))|(DFfilteredbindingitems.title.contains("Nathan Never"))|(DFfilteredbindingitems.title.contains("Zagor")))

Read Anobii data
+---------------+-------+----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+---------+-----------+-----------+
|item_content_id|item_id|source          |content                                                                                                                                                                                                      |language|preferred|provider_id|last_update|
+---------------+-------+----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+---------+-----------+-----------+
|1427528        |2539173|Book Description|Al mercato del villaggio si possono acquistare animali di tutti i

In [2]:
#DFfilteredmagazineitems.show(20, False)

#FILTRAGGIO 4: libri con più di X votes. Per vedere i voti di ogni libro dovrei usare il dataframe link_person_item.csv.
#Come possiamo notare, alcune recensione hanno valore 0: questo non vuol dire che il libro è stato valutato
#atrocemente ma che l'utente ha letto il libro senza inserire voti.

#Dobbiamo dunque scremare tali voti da questo dataframe.

# file = "/data/SMARTDATA/books/anobii/link_person_item.csv" #MOSTRA IL VOTO DI UN UTENTE A UN LIBRO
# DFstars = spark.read.csv(file,header=True)
#DFstars.show(50, False)

DFstarsfilteredno0 = DFinfo.filter(DFinfo.item_review > 0)
#DFstarsfilteredno0.show(20,False)

#Contiamo i libri con più voti e visualizziamoli in ordine discendente

DFgrouped = DFstarsfilteredno0.groupby("item_id").count().withColumnRenamed("count", "total_votes")\
                              .sort("total_votes", ascending=False)
#I voti dei libri più votati si aggirano intorno ai 10^4 come ordine di grandezza per cui per ora scelgo 300 come
#soglia di voti necessaria a rimanere nel dataframe (soggetto a cambiamenti)
DFgroupedfiltered = DFgrouped.filter(DFgrouped['total_votes'] > 300)
DFjoinbookstars = DFfilteredmagazineitems.join(DFgroupedfiltered, DFfilteredmagazineitems.item_id == DFgroupedfiltered.item_id).drop(DFgroupedfiltered.item_id)
# DFjoinbookstars.show(1, False)
# DFjoinbookstars:
# +-------+----------+-----------------+---------+----------+----------------+---------+---------+-------+----------------+------------+------------+------------------+--------+--------------+-----------+
# |item_id|isbn      |title            |sub_title|no_of_page|publication_date|publisher|binding  |edition|average_rating  |total_review|product_type|encrypt_item_id   |language|total_wishlist|total_votes|
# +-------+----------+-----------------+---------+----------+----------------+---------+---------+-------+----------------+------------+------------+------------------+--------+--------------+-----------+
# |1981748|8845260372|Giulietta squeenz|null     |209       |2008-05-01      |Bompiani |Paperback|null   |3.18773234200743|129         |1           |014f24ec9629744c88|11      |110           |543        |
# +-------+----------+-----------------+---------+----------+----------------+---------+---------+-------+----------------+------------+------------+------------------+--------+--------------+-----------+

#FILTRAGGIO 5: Pulizie varie ed eventuali
#DFfilteredlessthan10items.filter(DFfilteredlessthan10items.total_review > 100).show()

#NOTA: prendo i titoli con le minuscole per facilità di merging con le biblioteche
DFmanifestations_definitive_anobii = DFjoinbookstars.select("item_id", \
                                                            lower(DFjoinbookstars.title), \
                                                            lower(DFjoinbookstars.sub_title), \
                                                            "isbn",\
                                                            "average_rating", \
                                                            "total_review", \
                                                            "total_wishlist", \
                                                            "no_of_page",
                                                            "publication_date",
                                                            "publisher",
                                                            "binding",
                                                            "edition",
                                                            "product_type",
                                                            "total_votes",
                                                            "encrypt_item_id",)\
                                                    .withColumnRenamed("lower(title)", "title")\
                                                    .withColumnRenamed("lower(sub_title)", "sub_title")
# DFmanifestations_definitive_anobii.show(1, False)
# DFmanifestations_definitive_anobii: (uguale a DFjoinbookstars ma senza language)
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+
# |item_id|title            |sub_title|isbn      |average_rating  |total_review|total_wishlist|no_of_page|publication_date|publisher|binding  |edition|product_type|total_votes|encrypt_item_id   |
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+
# |1981748|giulietta squeenz|null     |8845260372|3.18773234200743|129         |110           |209       |2008-05-01      |Bompiani |Paperback|null   |1           |543        |014f24ec9629744c88|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+

print("Anobii data have been filtered (select books in italian, drop comics and books with a low number of ratings)")
print("Integrete with genre and author attributes Anobii data")
#add genres to the item table: this is contained in DFgenres:
# +---+--------+-------+----------+------------------+------------------+----------+-----+
# |id |familyid|itemid |categoryid|slug              |name              |languageid|votes|
# +---+--------+-------+----------+------------------+------------------+----------+-----+
# |1  |32423317|2823074|3         |business-economics|Business&Economics|3         |2.0  |
# +---+--------+-------+----------+------------------+------------------+----------+-----+
# Notice that if a book has more than 1 genre, DFgenres contained more than one row with that item_id, one for genre.
# Aggregation of rows with same itemid is needed 
DFgenres_aggregated = DFgenres.groupBy('itemid')\
                              .agg(F.concat_ws(" / ", F.collect_list('name'))\
                                    .alias('genres'))
# DFgenres_aggregated:
# +-------+----------------------------------------------------------------------------------------------+
# |itemid |genres                                                                                        |
# +-------+----------------------------------------------------------------------------------------------+
# |1981748|Humor / Fiction&Literature / Romance / Family-Sex&Relationships / Teens / Philosophy / History|
# +-------+----------------------------------------------------------------------------------------------+

# join between items and DFgenres_aggregated to have, for each item, its genres
DFmanifestations_definitive_anobii_genres = DFmanifestations_definitive_anobii.join(DFgenres_aggregated,\
                                                                                    DFgenres_aggregated.itemid == DFmanifestations_definitive_anobii.item_id)\
                                                                              .select(DFmanifestations_definitive_anobii["*"],\
                                                                                      DFgenres_aggregated["genres"]) 
# DFmanifestations_definitive_anobii_genres.show(1, False)
# DFmanifestations_definitive_anobii_genres:
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+
# |item_id|title            |sub_title|isbn      |average_rating  |total_review|total_wishlist|no_of_page|publication_date|publisher|binding  |edition|product_type|total_votes|encrypt_item_id   |genres                                                                                        |
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+
# |1981748|giulietta squeenz|null     |8845260372|3.18773234200743|129         |110           |209       |2008-05-01      |Bompiani |Paperback|null   |1           |543        |014f24ec9629744c88|History / Fiction&Literature / Family-Sex&Relationships / Teens / Humor / Romance / Philosophy|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+
# add a column which explain the origin of the data (anobii or BCT)
DFmanifestations_definitive_anobii_genres = DFmanifestations_definitive_anobii_genres.withColumn('data_type', lit("anobii"))
# DFmanifestations_definitive_anobii_genres.show(1, False)
# DFmanifestations_definitive_anobii_genres:
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+---------+
# |item_id|title            |sub_title|isbn      |average_rating  |total_review|total_wishlist|no_of_page|publication_date|publisher|binding  |edition|product_type|total_votes|encrypt_item_id   |genres                                                                                        |data_type|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+---------+
# |1981748|giulietta squeenz|null     |8845260372|3.18773234200743|129         |110           |209       |2008-05-01      |Bompiani |Paperback|null   |1           |543        |014f24ec9629744c88|Family-Sex&Relationships / Teens / Humor / Romance / Philosophy / History / Fiction&Literature|anobii   |
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+----------------------------------------------------------------------------------------------+---------+
# add author in the DFmanifestations_definitive_anobii_genres table
# find author_id for each book
DFmanifestations_anobii_genres_author = DFmanifestations_definitive_anobii_genres.join(DFauthorbooks, \
                                                                                       DFauthorbooks.item_id == DFmanifestations_definitive_anobii_genres.item_id)\
                                                                                 .select(DFmanifestations_definitive_anobii_genres["*"], \
                                                                                         DFauthorbooks["author_id"])
# DFmanifestations_anobii_genres_author.show(1, False)
# DFmanifestations_anobii_genres_author:
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+---------+
# |item_id|            title|sub_title|      isbn|  average_rating|total_review|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|   encrypt_item_id|              genres|data_type|author_id|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+---------+
# |1981748|giulietta squeenz|     null|8845260372|3.18773234200743|         129|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|014f24ec9629744c88|Romance / Humor /...|   anobii|   354644|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+---------+
# find author (name) for each book (and remove books which has same title but different auhtor?)
DFmanifestations_anobii_genres_author = DFmanifestations_anobii_genres_author.join(DFdisplay,\
                                                                                   DFmanifestations_anobii_genres_author.author_id == DFdisplay.author_id)\
                                                                             .select(DFmanifestations_anobii_genres_author["*"], \
                                                                                     DFdisplay["author_name"])\
                                                                             .drop("author_id")

# remove books which has same title but different auhtor
DFmanifestations_anobii_genres_author = DFmanifestations_anobii_genres_author.select("*", F.min(DFmanifestations_anobii_genres_author.author_name)\
                                                                                           .over(Window.partitionBy(DFmanifestations_anobii_genres_author.item_id))\
                                                                                           .alias("author"))\
                                                                             .drop("author_name")\
                                                                             .dropDuplicates(['item_id', 'author'])
# DFmanifestations_anobii_genres_author.show(1, False)
# DFmanifestations_anobii_genres_author:
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+
# |item_id|            title|sub_title|      isbn|  average_rating|total_review|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|   encrypt_item_id|              genres|data_type|    author|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+
# |1981748|giulietta squeenz|     null|8845260372|3.18773234200743|         129|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|014f24ec9629744c88|Philosophy / Hist...|   anobii|Pulsatilla|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+
#join of item and rating tables to obtain person_id, item_id, rating and all attributes of the item in the same row
#df1.join(df2, df1.id == df2.id).select(df1["*"],df2["other"])
DFloans_definitive_anobii_genres = DFstarsfilteredno0.join(DFmanifestations_anobii_genres_author, \
                                                           DFmanifestations_anobii_genres_author.item_id == DFstarsfilteredno0.item_id)\
                                                     .select(DFmanifestations_anobii_genres_author["*"], \
                                                             DFstarsfilteredno0["person_id"], \
                                                             DFstarsfilteredno0["item_review"])
# DFloans_definitive_anobii_genres.show(1, False)
# DFloans_definitive_anobii_genres:
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+
# |item_id|            title|sub_title|      isbn|  average_rating|total_review|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|   encrypt_item_id|              genres|data_type|    author|person_id|item_review|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+
# |1981748|giulietta squeenz|     null|8845260372|3.18773234200743|         129|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|014f24ec9629744c88|Philosophy / Fict...|   anobii|Pulsatilla|  1244593|          3|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+

print("Anobii data have been integreted with genre and author attributes")
print("Read and filter BCT data")
%run cleanDataBCTwithIBSN.ipynb #Così a quanto pare
DFmanifestations_definitive_bct = DFmanifestations_definitive
DFloans_definitive_bct = DFloans_definitive
# DFmanifestations_definitive_bct.show(1, False)
# DFmanifestations_definitive_bct:
# +----------------+------------+---------+-------------+---------+--------------------+------------+
# |edition_language|edition_date|    title|       author|publisher|manifestation_id_new|ISBNISSN_new|
# +----------------+------------+---------+-------------+---------+--------------------+------------+
# |             ita|        1996|Pinocchio|Carlo Collodi|   Nuages|              107930|  8807820714|
# +----------------+------------+---------+-------------+---------+--------------------+------------+
# DFloans_definitive_bct:
# +--------------------+--------------------+-------------------+-------------------+-------------------+------------+----------+-----------+
# |manifestation_id_new|       patron_id_md5|    loan_date_begin|      loan_date_end|           due_date|from_library|to_library|end_library|
# +--------------------+--------------------+-------------------+-------------------+-------------------+------------+----------+-----------+
# |              220771|fa242f1458100dccc...|2012-09-05 11:27:25|2012-09-27 18:18:00|2012-10-05 11:27:21|          18|        18|         18|
# +--------------------+--------------------+-------------------+-------------------+-------------------+------------+----------+-----------+
print("BCT data have been read and filtered")
print("Prepare data for merge")



Anobii data have been filtered (select books in italian, drop comics and books with a low number of ratings)
Integrete with genre and author attributes Anobii data
Anobii data have been integreted with genre and author attributes
Read and filter BCT data
Togli oggetti inutili: Prima 290125  - Dopo 257054
Togli libri in lingua straniera: Prima 290125  - Dopo 228059
Togli Fumetti: Prima 290125  - Dopo 227749
prefiltrati: 218419 - filtrati: 17947
Manifestation prefiltrati: 218419 - filtrati: 18164
Item prefiltrati: 998403 - filtrati: 217405
Loan prefiltrati: 5484078 - filtrati: 2172971
i libri sono: 18164
BCT data have been read and filtered
Prepare data for merge


In [3]:
# BCT data preparation: prepare the table of loans which contains also metadata of the book
DFloans_with_titles = DFloans_definitive_bct.join(DFmanifestations_definitive_bct, \
                                                  DFloans_definitive_bct.manifestation_id_new == DFmanifestations_definitive_bct.manifestation_id_new)\
                                             .select(DFloans_definitive_bct.manifestation_id_new, \
                                                     "patron_id_md5", \
                                                     "author", \
                                                     "ISBNISSN_new", \
                                                     lower(DFmanifestations_definitive_bct.title))\
                                             .withColumnRenamed("lower(title)", "title")
# DFloans_with_titles:
# +--------------------+--------------------+-------------+------------+---------+
# |manifestation_id_new|       patron_id_md5|       author|ISBNISSN_new|    title|
# +--------------------+--------------------+-------------+------------+---------+
# |              107930|b5c0986c79b1afafd...|Carlo Collodi|  8807820714|pinocchio|
# +--------------------+--------------------+-------------+------------+---------+

# BCT data preparation: split title and subtitle
RDDloans_with_titles = DFloans_with_titles.rdd    
RDDloans_for_merge = RDDloans_with_titles.map(title_and_subtitle)  
# DFloans_with_titles.show(1, False)
DFloans_to_merge = RDDloans_for_merge.toDF(["item_id", \
                                            "person_id", \
                                            "title", \
                                            "author", \
                                            "sub_title", \
                                            "item_review", \
                                            "total_review", \
                                            "average_rating", \
                                            "total_votes", \
                                            "isbn", \
                                            "data_type"])
# BCT data preparation: add missing columns, which are present in DFloans_definitive_anobii_genres
columns_to_add = ['total_wishlist',
                  'no_of_page',
                  'publication_date',
                  'publisher',
                  'binding',
                  'edition',
                  'product_type',
                  'encrypt_item_id',
                  'genre',
                 ]
values_to_add = {'total_wishlist': None,
                  'no_of_page': None,
                  'publication_date': None, 
                  'publisher': None, 
                  'binding': "Paperback",
                  'edition': None,
                  'product_type': 1,
                  'encrypt_item_id': None,
                  'genre': None,
    
}
for c in columns_to_add:
        DFloans_to_merge = DFloans_to_merge.withColumn(c, lit(values_to_add[c]))

# DFloans_to_merge:
# +-------+--------------------+---------+-----------+-------------+-----------+------------+--------------+-----------+----------+-----------+--------------+----------+----------------+---------+---------+-------+------------+---------------+-----+
# |item_id|           person_id|    title|author_name|    sub_title|item_review|total_review|average_rating|total_votes|      isbn|  data_type|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|encrypt_item_id|genre|
# +-------+--------------------+---------+-----------+-------------+-----------+------------+--------------+-----------+----------+-----------+--------------+----------+----------------+---------+---------+-------+------------+---------------+-----+
# | 107930|fcc17388ef20567fe...|pinocchio|       None|Carlo Collodi|          4|        None|          None|       None|8807820714|biblioteche|          null|      null|            null|     null|Paperback|   null|           1|           null| null|
# +-------+--------------------+---------+-----------+-------------+-----------+------------+--------------+-----------+----------+-----------+--------------+----------+----------------+---------+---------+-------+------------+---------------+-----+
print("Data have been prepared for merge")
print("Merge data")
# union of the loans of bct data and anobii data
DFloans_merged = DFloans_definitive_anobii_genres.union(DFloans_to_merge)
# DFloans_merged.show(1)
# DFloans_merged
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+
# |item_id|            title|sub_title|      isbn|  average_rating|total_review|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|   encrypt_item_id|              genres|data_type|    author|person_id|item_review|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+
# |1981748|giulietta squeenz|     null|8845260372|3.18773234200743|         129|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|014f24ec9629744c88|Family-Sex&Relati...|   anobii|Pulsatilla|   734393|          5|
# +-------+-----------------+---------+----------+----------------+------------+--------------+----------+----------------+---------+---------+-------+------------+-----------+------------------+--------------------+---------+----------+---------+-----------+
# Remove books, whose title is one word only, i.e. keep books whose title contains at least a blank space
# DFloans_merged.filter(DFloans_merged.title == "pinocchio").show()
DFloans_merged_nosinglewordtitles = DFloans_merged.filter(DFloans_merged.title.contains(" ")) 
# DFloans_merged_nosinglewordtitles.filter(DFloans_merged_nosinglewordtitles.title == "pinocchio").show()

# Assign to books with the same title, the same book_id
DFloans_merged_aggregated = DFloans_merged_nosinglewordtitles.select("*", \
                                                                     F.min(DFloans_merged_nosinglewordtitles.item_id)\
                                                                      .over(Window.partitionBy(DFloans_merged_nosinglewordtitles.title))\
                                                                      .alias("book_id"))\
                                                              .drop("item_id")

Data have been prepared for merge
Merge data


In [8]:
# DFloans_merged_aggregated.show(1)
# DFloans_merged_aggregated:
# +--------------------+-------------------+----------+----------------+------------+--------------+----------+----------------+-------------------+---------+-------+------------+-----------+------------------+--------------------+---------+----------------+---------+-----------+-------+
# |               title|          sub_title|      isbn|  average_rating|total_review|total_wishlist|no_of_page|publication_date|          publisher|  binding|edition|product_type|total_votes|   encrypt_item_id|              genres|data_type|          author|person_id|item_review|book_id|
# +--------------------+-------------------+----------+----------------+------------+--------------+----------+----------------+-------------------+---------+-------+------------+-----------+------------------+--------------------+---------+----------------+---------+-----------+-------+
# |come mi batte for...|storia di mio padre|8806198882|4.14122681883024|         206|           270|       302|      2009-11-03|Einaudi (Frontiere)|Hardcover|      1|           1|        705|01500978c278d06718|Crime / Teens / N...|   anobii|Benedetta Tobagi|   767565|          3|2806310|
# +--------------------+-------------------+----------+----------------+------------+--------------+----------+----------------+-------------------+---------+-------+------------+-----------+------------------+--------------------+---------+----------------+---------+-----------+-------+

# Assign to books with the same title, the same new_author
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.min(DFloans_merged_aggregated.author)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_author"))

# Assign to books with the same title, the same new_genre
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.min(DFloans_merged_aggregated.genre)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_genre"))

# Assign to books with the same title, the same new_encrypt
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.min(DFloans_merged_aggregated.encrypt_item_id)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_encrypt_item_id"))

# Assign to books with the same title, the same new_isbn
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.min(DFloans_merged_aggregated.isbn)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_isbn"))

# Update metrics of loans: new_total_count (to count the number of votes or loans) 
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.count("*")\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_total_count"))

# Update metrics of loans: new_average_rating 
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.avg(DFloans_merged_aggregated.item_review)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_average_rating"))

# Update metrics of loans: new_total_review, to count the actual number of review for a book (only anobii provides real review!!) 
DFloans_merged_aggregated = DFloans_merged_aggregated.select("*", \
                                                             F.max(DFloans_merged_aggregated.total_review)\
                                                              .over(Window.partitionBy(DFloans_merged_aggregated.book_id))\
                                                              .alias("new_total_review"))

# Remove "old" attributes
columnsToDrop = ['author',
                 'genres',
                 'encrypt_item_id',
                 'isbn',
                 'total_count',
                 'average_rating',
                 'total_review',]
DFloans_merged_aggregated = DFloans_merged_aggregated.drop(*columnsToDrop)
# DFloans_merged_aggregated.show(1)
# DFloans_merged_aggregated:
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+-------------------+----------+---------------+------------------+----------------+
# |            title|sub_title|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|data_type|person_id|item_review|book_id|new_author|           new_genre|new_encrypt_item_id|  new_isbn|new_total_count|new_average_rating|new_total_review|
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+-------------------+----------+---------------+------------------+----------------+
# |giulietta squeenz|     null|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|   anobii|   959028|          3|1981748|Pulsatilla|Philosophy / Hist...| 014f24ec9629744c88|8845260372|            543|3.1860036832412524|             129|
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+-------------------+----------+---------------+------------------+----------------+

# Rename "new" attributes
columnsToRenames = ['new_author',
                    'new_genre',
                    'new_encrypt_item_id',
                    'new_isbn',
                    'new_total_count',
                    'new_average_rating',
                    'new_total_review',]
for c in columnsToRenames:
    DFloans_merged_aggregated = DFloans_merged_aggregated.withColumnRenamed(c, c.replace("new_", ""))
# DFloans_merged_aggregated.show(1)
# DFloans_merged_aggregated:
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+------------------+----------+-----------+------------------+------------+
# |            title|sub_title|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|data_type|person_id|item_review|book_id|    author|               genre|   encrypt_item_id|      isbn|total_count|    average_rating|total_review|
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+------------------+----------+-----------+------------------+------------+
# |giulietta squeenz|     null|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|   anobii|   273238|          4|1981748|Pulsatilla|Humor / Philosoph...|014f24ec9629744c88|8845260372|        543|3.1860036832412524|         129|
# +-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+---------+-----------+-------+----------+--------------------+------------------+----------+-----------+------------------+------------+

# Remove ['book_id', 'person_id'] duplicates
DFloans_merged_aggregated = DFloans_merged_aggregated.dropDuplicates(['book_id', 'person_id'])
print("Data has been merged")
# before write data on file, create index for both users and books of the sparce matrix
RDDmerged_almost_definitive = DFloans_merged_aggregated.rdd

# functions which creates the dictionary where the key is the book_id/person_id and the value is the corresponding index
book_dictionary = create_book_dictionary(RDDmerged_almost_definitive)
user_dictionary = create_user_dictionary(RDDmerged_almost_definitive)

# add the user_index and the book_index in the dataframe
RDDdefinitive = RDDmerged_almost_definitive.map(addRowColumnId)
DFloans_merged_aggregated = RDDdefinitive.toDF(['book_id',
                                                'title',
                                                'sub_title',
                                                'total_wishlist',
                                                'no_of_page',
                                                'publication_date',
                                                'publisher',
                                                'binding',
                                                'edition',
                                                'product_type',
                                                'total_votes',
                                                'data_type',
                                                'person_id',
                                                'item_review',
                                                'author',
                                                'genre',
                                                'encrypt_item_id',
                                                'isbn',
                                                'total_count',
                                                'average_rating',
                                                'total_review', 
                                                'user_index', 
                                                'book_index'], sampleRatio=0.9)
print("Write data on csv files")
ratings = DFloans_merged_aggregated.filter((DFloans_merged_aggregated.genre != "null"))#DFdefinitive

# in the rating table, each row is given by "person_id", "book_id", "rating" attributes
ratings_tocsv = ratings.select(["person_id", "book_id", "item_review"])\
                       .withColumnRenamed("item_review", "rating")
# ratings_tocsv.show(1)
# ratings_tocsv:
# +---------+-------+------+
# |person_id|book_id|rating|
# +---------+-------+------+
# |   100489|1981748|     3|
# +---------+-------+------+
print("There are "+str(ratings_tocsv.count())+" loans")
ratings_tocsv.toPandas().to_csv("ratings2021.csv")

# in the book table, the metadata about the book are reported
books_tocsv = ratings.dropDuplicates(['book_id'])\
                     .drop("person_id")\
                     .drop("user_index")\
                     .drop("item_review")
# books_tocsv.show(1)
# books_tocsv:
# +-------+-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+----------+--------------------+------------------+----------+-----------+------------------+------------+----------+
# |book_id|            title|sub_title|total_wishlist|no_of_page|publication_date|publisher|  binding|edition|product_type|total_votes|data_type|    author|               genre|   encrypt_item_id|      isbn|total_count|    average_rating|total_review|book_index|
# +-------+-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+----------+--------------------+------------------+----------+-----------+------------------+------------+----------+
# |1981748|giulietta squeenz|     null|           110|       209|      2008-05-01| Bompiani|Paperback|   null|           1|        543|   anobii|Pulsatilla|Family-Sex&Relati...|014f24ec9629744c88|8845260372|        543|3.1860036832412524|         129|       319|
# +-------+-----------------+---------+--------------+----------+----------------+---------+---------+-------+------------+-----------+---------+----------+--------------------+------------------+----------+-----------+------------------+------------+----------+
print("There are "+str(books_tocsv.count())+" books")

books_tocsv = books_tocsv.join(DFdescriptions, DFdescriptions.item_id == books_tocsv.book_id, "left").select("book_id", "title", "sub_title", "total_wishlist", "no_of_page", "publication_date", "publisher", "binding", "edition", "product_type", "total_votes", "data_type", "author", "genre", "encrypt_item_id", "isbn", "total_count", "average_rating", "total_review", "book_index", "content")
books_tocsv.toPandas().to_csv("books2021.csv")

# in the user table, the person_id is reported for each user (in each row)
users_tocsv = ratings.select("person_id", "user_index").dropDuplicates()
# users_tocsv.show(1)
# users_tocsv:
# +---------+----------+
# |person_id|user_index|
# +---------+----------+
# |  1224803|     10623|
# +---------+----------+
print("There are "+str(users_tocsv.count())+" users")
users_tocsv.toPandas().to_csv("users2021.csv")

print("Csv files have been written")

Data has been merged
Write data on csv files
There are 3812993 loans
There are 3045 books
There are 144202 users
Csv files have been written


In [10]:
###ESTRAZIONE KEYWORD###

#Usiamo una versione filtrata di books_tocsv dove non compaiono tuple senza descrizione per attuare la nostra
#estrazione.

books_with_content = books_tocsv.filter(books_tocsv.content!="")
#books_with_content.count() #1387 libri



1387

In [39]:
#Per estrarre le keyword in maniera "TF-IDF" dobbiamo innanzitutto implementare una funzione che pulisca il testo 
#dalle stopwords (congiunzioni, rumori, etc.)

import re 

def clean_text(line):
    description = line[20]
    description = description.replace("<b>", " ")
    description = description.replace("</b>", " ")
    description = description.replace("<br>", " ")
    description = description.replace("</br>", " ")
    description = description.replace("<p>", " ")
    description = description.replace("</p>", " ")
    description = description.replace("<P>", " ")
    description = description.replace("</P>", " ")
    description = description.replace("<i>", " ")
    description = description.replace("</i>", " ")
    description = description.replace("&quot", " ")
    description = description.replace("<strong>", " ")
    description = description.replace("</strong>", " ")
    pattern = re.compile(re.escape(' e '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' o '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' il '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' lo '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' la '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' i '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' gli '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' le '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' di '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' a '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' da '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' in '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' con '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' per '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' su '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' tra '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' fra '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' degli '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' del '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' delle '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' dello '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' della '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' agli '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' al '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' alle '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' alla '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' allo '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' un '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' uno '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' una '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' che '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' ma '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' però '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' perché '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' perchè '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' non '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' ad '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' nella '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' nello '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' nelle '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' negli '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' sulle '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' sugli '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' sulla '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' sullo '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' del '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' sul '), re.IGNORECASE)
    description = pattern.sub('', description)
    pattern = re.compile(re.escape(' nel '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' al '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(' si '), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape("all'"), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape("nell'"), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape("dell'"), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape("sull'"), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape("l'"), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" mio "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" tuo "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" suo "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" mia "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" tua "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" sua "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" nostro "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" vostro "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" loro "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" nostra "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" vostra "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" io "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" tu "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" lui "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" lei "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" noi "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" voi "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    pattern = re.compile(re.escape(" essi "), re.IGNORECASE)
    description = pattern.sub(' ', description)
    description = description.replace(",", " ")
    description = description.replace(".", " ")
    description = description.replace('"', " ")
    description = description.replace(";", " ")
    description = description.replace(":", " ")
    description = description.replace("!", " ")
    description = description.replace("?", " ")
    description = description.replace(" è ", " ")
    description = description.replace(" é ", " ")
    description = description.replace(" o ", " ")
    description = description.replace(" ' ", " ")
    description = description.replace("  ", " ")
    
    return Row(book_id=line[0], title=line[1], sub_title=line[2], 
               total_wishlist=line[3], no_of_page=line[4], publication_date=line[5], 
               publisher=line[6], binding=line[7], edition=line[8], product_type=line[9], 
               total_votes=line[10], data_type=line[11], author=line[12], genre=line[13], 
               encrypt_item_id=line[14], isbn=line[15], total_count=line[16], average_rating=line[17], 
               total_review=line[18], book_index=line[19], content=description)
RDDcontents = books_with_content.rdd
        
###SCHEMA###
#"book_id", "title", "sub_title", "total_wishlist", "no_of_page", "publication_date", 
#"publisher", "binding", "edition", "product_type", "total_votes", "data_type", "author", 
#"genre", "encrypt_item_id", "isbn", "total_count", "average_rating", "total_review", "book_index", "content"
#print(RDDcontents.take(1))
RDDcleaned = RDDcontents.map(clean_text)
#print(RDDcleaned.take(1))

DFcleaned = RDDcleaned.toDF()

In [41]:
#Ora che abbiamo il dataframe pulito, convertiamolo in pandas per estrarre i documenti da utilizzare

DFcleaned_pandas = DFcleaned.toPandas()
#DFcleaned_pandas.dropna(subset=['content'], inplace=True)
#DFcleaned_pandas.head()

corpus_doc = DFcleaned_pandas['content'].to_list()
#print(corpus_doc) #Lista di tutte le descrizioni del dataset

In [52]:
#Importiamo il TFIDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(smooth_idf=True, use_idf=True)

vectorizer.fit_transform(corpus_doc) #CREAZIONE DIZIONARIO CON I VOCABOLI CONTENUTI NELLE DESCRIZIONI

feature_names = vectorizer.get_feature_names() #Estrae le feature (i vocaboli) importanti

def sort_coo(coo_matrix): #Funzione usata per riordinare il dizionario per score
    tuples = zip(coo_matrix.col, coo_matrix.data) #COLONNE: FEATURES RIGHE: DESCRIZIONE
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10): #Funzione usata per ottenere le n parole chiave
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    
    for idx, score in sorted_items:
    
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

def get_keywords(vectorizer, feature_names, doc): #Funzione usata per ritornare le top k parole chiave di una descrizione
    tf_idf_vector = vectorizer.transform([doc])
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    keywords=extract_topn_from_vector(feature_names,sorted_items,10) #10 keyword per ora
    
    return list(keywords.keys())

result = []
for descrizione in corpus_doc:
    df = {}
    df['content'] = descrizione
    df['top_keywords'] = get_keywords(vectorizer, feature_names, descrizione)
    result.append(df)
    
DFkeywords = pd.DataFrame(result)
DFkeywords

,content,top_keywords
0,Giulietta bambina intelligente insubordinata a...,"[decide, visivo, simpatizzando, sfasciafamigli..."
1,Allegra sparita Torino c'è più meglio c'è anc...,"[iaio, allegra, zombi, ville, troppa, testimon..."
2,Togliere mezzo Robert Kelly anziano solitario...,"[kelly, cambia, film, zane, village, videoteca..."
3,Valter burla mondo sempre abituato perdere Pen...,"[dolore, valter, scusa, goccia, gioia, sempre,..."
4,Clay Easton diventato sceneggiatore proprio su...,"[clay, turner, rip, julian, molto, diventato, ..."
...,...,...
1382,Pastorale americana soltanto un'allegoria poli...,"[dettagli, zuckerman, pastorale, intollerabili..."
1383,Che cosa terribile Purtroppo libreria Il Pap...,"[verificato, todorovic, papiro, purtroppo, tri..."
1384,Anita Blake vorrebbe assistere pace nozze amic...,"[anita, vampiri, master, claude, jean, tramand..."
1385,Chiunque abbia visto qualche film terrore cent...,"[vance, eleanor, scegliere, turno, torrette, p..."


In [62]:
#Ora che abbiamo il dataframe di pandas con le keywords associate alle descrizioni, non ci resta che joinare
#il contenuto di esso con il dataframe dei libri, integrando l'attributo keywords.
#Riportiamo il dataframe nella forma PySpark

#DFkeywords = spark.createDataFrame(DFkeywords)

DF_almost_final = DFcleaned.join(DFkeywords, DFkeywords.content == DFcleaned.content, "left").drop(DFkeywords.content).select("book_id", "top_keywords")
#DF_almost_final.show(1, False)
#Adesso joiniamo per book_id al dataframe con più libri (compresi quelli senza descrizione)
DF_final = books_tocsv.join(DF_almost_final, DF_almost_final.book_id == books_tocsv.book_id, "left").drop(DF_almost_final.book_id)

DF_final.toPandas().to_csv('books_and_keywords.csv')